In [119]:
import torch
from torch import nn
import numpy as np
import pandas as pd
from torch.nn import functional as F
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [120]:
data = pd.read_csv("LSTM-Multivariate_pollution.csv",index_col = 0)
#data['date'] = pd.to_datetime(data['date'],format='%Y-%m-%d %H:%M:%S')

In [121]:
data.dtypes

pollution    float64
dew            int64
temp         float64
press        float64
wnd_dir       object
wnd_spd      float64
snow           int64
rain           int64
dtype: object

In [122]:
data.head()

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
date,,,,,,,,
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0


In [123]:
wind_dir_enc = LabelEncoder()
data["wnd_dir"]= wind_dir_enc.fit_transform(data["wnd_dir"])
data["wnd_dir"] = data["wnd_dir"].astype(float)

In [124]:
values = data.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [125]:
scaled

array([[0.12977867, 0.35294118, 0.24590164, ..., 0.00229001, 0.        ,
        0.        ],
       [0.14889336, 0.36764706, 0.24590164, ..., 0.00381099, 0.        ,
        0.        ],
       [0.15995976, 0.42647059, 0.2295082 , ..., 0.00533197, 0.        ,
        0.        ],
       ...,
       [0.01006036, 0.26470588, 0.26229508, ..., 0.41399641, 0.        ,
        0.        ],
       [0.00804829, 0.26470588, 0.24590164, ..., 0.42086644, 0.        ,
        0.        ],
       [0.01207243, 0.27941176, 0.26229508, ..., 0.4262155 , 0.        ,
        0.        ]])

In [126]:
scaled.shape

(43800, 8)

In [127]:
data.isnull().any()

pollution    False
dew          False
temp         False
press        False
wnd_dir      False
wnd_spd      False
snow         False
rain         False
dtype: bool

### create sequence data

In [128]:
# split into train and test sets
n_train_hours = int(scaled.shape[0])*0.75
train = scaled[:int(n_train_hours), :]
test = scaled[int(n_train_hours):, :]

In [129]:
train.shape

(32850, 8)

In [130]:
test.shape

(10950, 8)

In [136]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1,0:1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq   

In [139]:
tw = 4
train_X = []
train_Y = []

train_in_out = create_inout_sequences(train,tw)
for h in range(len(train_in_out)):
    train_x = train_in_out[h][0]
    train_y = train_in_out[h][1]
        
    train_X.append(train_x)
    train_Y.append(train_y)
    
train_X_ = np.stack(train_X) 
train_Y_ = np.stack(train_Y).flatten()    


In [140]:
tw = 4
test_X = []
test_Y = []

test_in_out = create_inout_sequences(test,tw)
for h in range(len(test_in_out)):
    test_x = test_in_out[h][0]
    test_y = test_in_out[h][1]

    test_X.append(test_x)
    test_Y.append(test_y)
    
test_X_ = np.stack(test_X) 
test_Y_ = np.stack(test_Y).flatten()    

In [141]:
train_X_.shape

(32846, 4, 8)

In [143]:
train_Y_.shape

(32846,)

In [144]:
test_X_.shape

(10946, 4, 8)

In [145]:
test_Y_.shape

(10946,)

### define model

In [ ]:
class LSTM(nn.Module):
    def __init__(self,input_size,output_size,n_hidden = 128,n_layers = 2,
                drop_prob = 0.5,lr=0.001,train_on_gpu=False):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.lr = lr
        self.input_size = input_size
        self.output_size = output_size
        self.train_on_gpu = train_on_gpu
        
        self.lstm = nn.LSTM(input_size,n_hidden,n_layers,
                           dropout=drop_prob,batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden,output_size)
       
        
        
    def forward(self,x,hidden):
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)     
        out = out.contiguous().view(-1, self.n_hidden) 
        out = self.fc(out)
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        
        if (self.train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden